# Exchange - Single-producer / Multi-consumer Implementation

---

## Introduction

This single-producer, multi-consumer (SPMC) exchange architecture serializes all order book mutations through a dedicated command queue. A single producer thread is responsible for applying all state changes—such as order submissions and cancellations—while multiple consumers can concurrently read the order book. This design preserves the underlying OrderBook’s price–time priority and achieves constant-time $O(1)$ insert, cancel, and match operations. It effectively replicates the structure of a real exchange, where numerous participants interact with a shared market state while maintaining deterministic execution of order events.

To ensure safe concurrent access, the system employs a read–write lock. Because read operations do not alter shared state, multiple readers can hold the lock simultaneously without interference. Write operations, however, mutate the order book and therefore require exclusive access. When a writer acquires the lock, it must wait until all active readers have finished, and no new reader can enter until the write completes.

This locking strategy maximizes throughput under read-intensive workloads while guaranteeing data consistency during writes.

**Performance Note:**

This approach is well suited for trading simulators, research environments, and lower-frequency strategies. True high-frequency trading systems, however, demand lower latency and would leverage specialized concurrency models - such as the **LMAX Disruptor** pattern and mechanically sympathetic memory layouts - to minimize contention and maximize cache efficiency.


In [ ]:
import logging
import threading
from decimal import Decimal
from queue import Empty, Queue
from typing import Any, Dict, Optional, Tuple

import requests
import uvicorn
from fastapi import FastAPI
from importnb import Notebook
from pydantic import BaseModel, Field

with Notebook():
    from notebooks.finance.order_book.implementation.__intermediate__l3_order_book import (
        OrderBook,
        Side,
    )

In [ ]:
log = logging.getLogger(__name__)
log.setLevel(logging.INFO)
if not log.hasHandlers():
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("[%(asctime)s] [%(levelname)s] %(message)s"))
    log.addHandler(handler)

In [ ]:
class OrderIdAllocator:
    def __init__(self):
        self._lock = threading.Lock()
        self._id = 1

    def next_id(self) -> int:
        with self._lock:
            oid = self._id
            self._id += 1
        return oid

class SPMCExchange:
    def __init__(self):
        self.order_book = OrderBook()

        # Order ID allocation
        self.order_id_allocator = OrderIdAllocator()

        # Read-write concurrency control
        self.rwlock = ReaderWriterLock()

        # Single producer (mutation executor)
        self.mutation_queue = Queue(maxsize=1_000_000)
        self.producer_thread = threading.Thread(target=self._producer_loop, daemon=True)
        self.producer_thread.start()

    def _producer_loop(self):
        while True:
            try:
                func = self.mutation_queue.get_nowait()
                self.rwlock.acquire_write()
                func()
            except Empty:
                # No mutations to process, yield briefly
                threading.Event().wait(0.00001)
            except Exception as e:
                log.error(f"Error processing mutation: {e}")
            finally:
                self.rwlock.release_write()

    # --- Write operations (enqueue mutations) ---

    def submit_limit_order(
        self,
        participant_id: str,
        side: Side,
        price: Decimal,
        qty: Decimal,
    ) -> int:
        order_id = self.next_order_id()
        fn = lambda: self.order_book.insert(
            order_id=order_id,
            participant_id=participant_id,
            side=side,
            price=price,
            qty=qty,
        )
        self.mutation_queue.put_nowait(fn)
        return order_id

    def submit_market_order(
        self,
        participant_id: str,
        side: "Side",
        qty: Decimal,
    ) -> int:
        order_id = self.next_order_id()
        fn = lambda: self.order_book.insert(
            order_id=order_id,
            participant_id=participant_id,
            side=side,
            qty=qty,
        )
        self.mutation_queue.put_nowait(fn)
        return order_id

    def cancel_order(self, order_id: int):
        self.mutation_queue.put_nowait(lambda: self.order_book.cancel(order_id=order_id))

    # --- Read operations (direct, no queuing) ---

    def best_bid(self) -> Optional[Tuple[Decimal, Decimal]]:
        with self.rwlock.read_lock():
            return self.order_book.best_bid()

    def best_ask(self) -> Optional[Tuple[Decimal, Decimal]]:
        with self.rwlock.read_lock():
            return self.order_book.best_ask()

    def volume(self, price: Decimal) -> Dict["Side", Decimal]:
        with self.rwlock.read_lock():
            return self.order_book.get_volume(price)

    def orders(self, price: Decimal) -> Dict[int, Any]:
        with self.rwlock.read_lock():
            return self.order_book.get_orders_at_price(price)


## FastAPI Integration
The exchange can be exposed as a RESTful or WebSocket service using **FastAPI**, providing an asynchronous interface for order submission, cancellation, and market data queries. FastAPI’s  event loop (based on `uvicorn` and `asyncio`) allows multiple clients to interact with the system concurrently while the single-producer thread maintains deterministic sequencing of writes. This separation enables realistic simulation of exchange connectivity, client order flow, and streaming market data updates with minimal overhead.

In [ ]:
class OrderRequest(BaseModel):
    participant_id: str = Field(..., min_length=1, max_length=20)
    side: Side = Field(...)
    price: Decimal = Field(..., gt=0)
    qty: Decimal = Field(..., gt=0)

class CancelRequest(BaseModel):
    order_id: int = Field(..., gt=0)


In [ ]:
app = FastAPI(
    title="SPMC Exchange API",
    description="Single Producer Multi Consumer Exchange",
    version="1.0.0"
)

exchange = SPMCExchange()

@app.post("/orders/limit", status_code=201, response_model=dict)
async def submit_limit_order(order: OrderRequest):
    oid = exchange.submit_limit_order(
        order.participant_id,
        order.side,
        order.price,
        order.qty
    )
    return {"order_id": oid, "status": "accepted"}

@app.post("/orders/market", status_code=201, response_model=dict)
async def submit_market_order(order: OrderRequest):
    oid = exchange.submit_market_order(
        order.participant_id,
        order.side,
        order.qty
    )
    return {"order_id": oid, "status": "accepted"}

@app.delete("/orders/{order_id}", response_model=dict)
async def cancel_order(order_id: int):
    exchange.cancel_order(order_id)
    return {"order_id": order_id, "status": "cancel_sent"}

@app.get("/orders/{price}", response_model=Dict)
async def get_orders(price: Decimal):
    orders = exchange.orders(price)
    return {
        "price": str(price),
        "orders": {str(oid): {"participant_id": o.participant_id, "side": o.side.value, "qty": str(o.qty)} for oid, o in orders.items()}
    }

@app.get("/book/best_bid", response_model=Optional[Dict])
async def get_best_bid():
    result = exchange.best_bid()
    return {"price": str(result[0]), "size": int(result[1])} if result else None

@app.get("/book/best_ask", response_model=Optional[Dict])
async def get_best_ask():
    result = exchange.best_ask()
    return {"price": str(result[0]), "size": int(result[1])} if result else None

@app.get("/book/volume/{price}", response_model=Dict)
async def get_volume(price: Decimal):
    vol = exchange.volume_at_price(price)
    return {
        "price": str(price),
        "bid_volume": str(vol.get(Side.BID, Decimal(0))),
        "ask_volume": str(vol.get(Side.ASK, Decimal(0)))
    }

@app.get("/health")
async def health_check():
    """Health check endpoint"""
    return {"status": "healthy"}


In [ ]:
import time

if __name__ == "__main__":    
    threading.Thread(target=lambda: uvicorn.run(app, host="127.0.0.1", port=8000, log_level="error"), daemon=True).start()

    time.sleep(2)  # Give the server a moment to start
    
    response = requests.post("http://127.0.0.1:8000/orders/limit", 
                        json={"participant_id": "MM1", "side": "BID", 
                              "price": "99.50", "qty": "100"})
    print(response.json())
    
    response = requests.get("http://127.0.0.1:8000/book/best_bid")
    print(response.json())